# Models Training Status

In this notebook there are scripts to check which models are there and which are missing.

In [1]:
base_path = "/data/tbaldi/checkpoint/"
sizes = ['small', 'baseline', 'large']
batch_sizes = [16, 32, 64, 128, 256, 512, 1024]
learning_rates = [0.1, 0.05, 0.025, 0.0125, 0.00625, 0.003125, 0.0015625]
precisions = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [2]:
missing = []
def get_missing_econ(bs, lr, precision):
    missing = []
    for size in sizes:
        # print(f'BATCH SIZE: {bs} - LEARNING RATE {lr} - SIZE {size} - PRECISION {precision}')
        for exp in range(1, 4):
            file_path = base_path + f'bs{bs}_lr{lr}/ECON_{precision}b/{size}/{size}_emd_{exp}.txt'
            try:
                with open(file_path, 'r') as file:
                    content = file.read()

            except Exception as e:
                if size == 'small':
                    missing.append((bs, lr, precision))
                    continue
    return set(missing)


In [3]:
missing = {}
for bs in batch_sizes:
    
    print(f'=========== {bs} ==========')
    for lr in learning_rates:
        print(f'---------- {lr} ---------')
        for precision in precisions:
            mis = get_missing_econ(bs, lr, precision)
            if mis:
                print(f'{bs} - {lr} - {precision}')
            
missing

=========== 16 ==========
---------- 0.1 ---------
16 - 0.1 - 9
16 - 0.1 - 10
16 - 0.1 - 11
---------- 0.05 ---------
16 - 0.05 - 2
16 - 0.05 - 5
16 - 0.05 - 6
16 - 0.05 - 7
16 - 0.05 - 8
16 - 0.05 - 9
16 - 0.05 - 10
16 - 0.05 - 11
---------- 0.025 ---------
16 - 0.025 - 6
16 - 0.025 - 7
16 - 0.025 - 8
16 - 0.025 - 9
16 - 0.025 - 10
16 - 0.025 - 11
---------- 0.0125 ---------
16 - 0.0125 - 5
16 - 0.0125 - 6
16 - 0.0125 - 7
16 - 0.0125 - 8
16 - 0.0125 - 9
16 - 0.0125 - 10
16 - 0.0125 - 11
---------- 0.00625 ---------
16 - 0.00625 - 4
16 - 0.00625 - 5
16 - 0.00625 - 6
16 - 0.00625 - 7
16 - 0.00625 - 8
16 - 0.00625 - 9
16 - 0.00625 - 10
16 - 0.00625 - 11
---------- 0.003125 ---------
16 - 0.003125 - 2
16 - 0.003125 - 5
16 - 0.003125 - 6
16 - 0.003125 - 7
16 - 0.003125 - 8
16 - 0.003125 - 9
16 - 0.003125 - 10
16 - 0.003125 - 11
---------- 0.0015625 ---------
16 - 0.0015625 - 2
16 - 0.0015625 - 3
16 - 0.0015625 - 4
16 - 0.0015625 - 5
16 - 0.0015625 - 6
16 - 0.0015625 - 7
16 - 0.0015625 - 8


{}